# Contact Path Analysis

This notebook analyzes a contact between two cells, found by dilating them both by a few pixels and then identifying the region of overlap.  It's a 2D process since in the datasets I've been working with, the Z resolution is so poor that humans routinely work in a 2D view, and the point of all this is to identify putative synapses for human review.

Goals of the current analysis:
1. Find a good "midpoint" for a contact region (which always looks like a thick line).
2. Create a perpendicular line at that point.

When we draw this perpendicular, it should appear to bisect the line into two approximately equal halves.  (If the line is a loop, then it will cut at some arbitrary position on the loop.)  For our synapse application, this line will represent a putative synapse, connecting the presynaptic and postsynaptic cells at the center of the contact.

In [ ]:
# Imports
from zetta_utils.layer.volumetric.cloudvol import build_cv_layer
from zetta_utils.geometry import Vec3D
import cc3d
from collections import deque
import numpy as np
import zetta_utils.tensor_ops.convert as convert
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from skimage.morphology import skeletonize
import time

In [ ]:
# Constants
CONTACT_SEG_PATH = "gs://tmp_2w/joe/concact-20240801"
RESOLUTION = (8, 8, 42)  # (nm)
BOUNDS_START = (27991, 21266, 3063)  # (voxels)
BOUNDS_END = (28247, 21522, 3103)
Z = 3081  # particular Z level we want to work with

In [ ]:
# Load and extract the 2D image at the given Z
cvl = build_cv_layer(path = CONTACT_SEG_PATH)
data = cvl[RESOLUTION, BOUNDS_START[0]:BOUNDS_END[0], BOUNDS_START[1]:BOUNDS_END[1], Z:Z+1]
data = data[0, :, :, 0]
print(f'Loaded {data.shape} image of type {data.dtype}, with {len(np.unique(data))} unique values')

In [ ]:
# Let's plot the data so we can see what it looks like
def plot(data):
    unique_values = np.unique(data)
    qty_values = len(unique_values)
    
    # create appropriately sized color map
    if qty_values > 2:
        colors = plt.colormaps['tab20'](np.linspace(0, 1, qty_values - 1))
        color_list = [(0,0,0)] + [colors[i] for i in range(qty_values-1)]
        cmap = mcolors.ListedColormap(color_list)
    else:
        cmap = 'gray'
    
    # remap unique values to sequential values from 0 - qty_values-1
    value_to_index = {v: i for i, v in enumerate(unique_values)}
    indexed_data = np.vectorize(value_to_index.get)(data)

    # transpose to match the orientation seen in NG
    indexed_data = np.transpose(indexed_data)

    # plot the data using the indexed colors
    plt.figure(figsize=(4,4))
    plt.imshow(indexed_data, cmap=cmap, interpolation='nearest')
    plt.show()
plot(data)

In [ ]:
# Now let's focus on a single value.
CONTACT_ID = 70
contact_data = (data == CONTACT_ID).astype(bool)
plot(contact_data)

In [ ]:
# Skeletonize this contact region.
skeleton = skeletonize(contact_data)
plot(skeleton)

In [ ]:
# Let's find the endpoints of the skeleton.
# (For now, assume that there are exactly two such; ToDo: deal with gaps (more than 2 endpoints) and loops (0 endpoints).
def find_endpoints(skeleton):
    endpoints = []
    # Define the 8-connectivity structure
    struct = np.array([[1,1,1],
                       [1,0,1],
                       [1,1,1]])
    
    for i in range(1, skeleton.shape[0] - 1):
        for j in range(1, skeleton.shape[1] - 1):
            if skeleton[i, j] == 1:
                # Count neighbors
                neighbors = np.sum(skeleton[i-1:i+2, j-1:j+2] * struct)
                if neighbors == 1:
                    endpoints.append((i, j))
    return endpoints
endpoints = find_endpoints(skeleton)
print(f'Skeleton endpoints: {endpoints}')

In [ ]:
# Now trace the path, again using 8-neighbor connectivity.
def trace_path(skeleton, start):
    path = []
    queue = deque([start])
    visited = set()
    
    while queue:
        current = queue.popleft()
        if current in visited:
            continue
        visited.add(current)
        path.append(current)
        
        i, j = current
        # Look at all 8 neighbors
        for ni in range(i-1, i+2):
            for nj in range(j-1, j+2):
                if (ni, nj) != (i, j) and skeleton[ni, nj] == 1 and (ni, nj) not in visited:
                    queue.append((ni, nj))
                    break # Found the next step in the path
    return path
path = trace_path(skeleton, endpoints[0])
midpoint = path[len(path) // 2]
print(f'Traced path of length {len(path)} from {path[0]} to {path[-1]}, with midpoint at {midpoint}')

In [ ]:
# Estimate the slope at the midpoint, so we can find a perpendicular.

In [ ]:
def estimate_slope(path, midpoint, window=3):
    mid_index = path.index(midpoint)
    
    # Define indices for a small segment around the midpoint
    start_index = max(0, mid_index - window)
    end_index = min(len(path) - 1, mid_index + window)
    
    # Coordinates of the start and end points of the segment
    start_point = path[start_index]
    end_point = path[end_index]
    
    # Calculate the vector (dx, dy)
    dx = end_point[0] - start_point[0]
    dy = end_point[1] - start_point[1]
    
    # Normalize the vector to avoid scaling issues
    length = np.hypot(dx, dy)
    if length != 0:
        dx /= length
        dy /= length
    
    return dx, dy
dx, dy = estimate_slope(path, midpoint)
print(f'Slope at midpoint: {(dx, dy)}')

In [ ]:
# define a bisecting line (which could also be a putative synapse!)
hl = 10  # line half-length
line = ((midpoint[0]-dy*hl, midpoint[1]+dx*hl), (midpoint[0]+dy*hl, midpoint[1]-dx*hl), 'red')
print(f'Bisecting line: {line}')

In [ ]:
# Let's expand our plot function so it can include a set of lines.
# Each line defined as (xy0, xy1, color).
def plot(data, lines=[]):
    unique_values = np.unique(data)
    qty_values = len(unique_values)
    
    # create appropriately sized color map
    if qty_values > 2:
        colors = plt.colormaps['tab20'](np.linspace(0, 1, qty_values - 1))
        color_list = [(0,0,0)] + [colors[i] for i in range(qty_values-1)]
        cmap = mcolors.ListedColormap(color_list)
    else:
        cmap = 'gray'
    
    # remap unique values to sequential values from 0 - qty_values-1
    value_to_index = {v: i for i, v in enumerate(unique_values)}
    indexed_data = np.vectorize(value_to_index.get)(data)

    # transpose to match the orientation seen in NG
    indexed_data = np.transpose(indexed_data)

    # plot the data using the indexed colors
    plt.figure(figsize=(4,4))
    plt.imshow(indexed_data, cmap=cmap, interpolation='nearest')

    # additional lines: plot 'em if you got 'em
    for line in lines:
        (x0, y0), (x1, y1), color = line
        plt.plot([x0, x1], [y0, y1], color=color, linewidth=2)
    plt.show()
plot(contact_data, [line])

In [ ]:
# Let's wrap all that up into a single method, and try it on a couple other contacts.
def plot_bisection(data, contact_id):
    contact_data = (data == contact_id).astype(bool)
    skeleton = skeletonize(contact_data)
    endpoints = find_endpoints(skeleton)
    path = trace_path(skeleton, endpoints[0])
    midpoint = path[len(path) // 2]
    print(f'Traced path of length {len(path)} from {path[0]} to {path[-1]}, with midpoint at {midpoint}')
    dx, dy = estimate_slope(path, midpoint)
    hl = 10  # line half-length
    line = ((midpoint[0]-dy*hl, midpoint[1]+dx*hl), (midpoint[0]+dy*hl, midpoint[1]-dx*hl), 'red')
    plot(contact_data, [line])

In [ ]:
plot_bisection(data, 71)

In [ ]:
plot_bisection(data, 108)

In [ ]:
plot_bisection(data, 67)

## Future Work

- The path extraction probably does not deal well with gaps.  When we have more than two endpoints, we might need to try tracing the path from each of them, and pick the longest path for bisection.
- The last test above does seem to handle a loop acceptably, but it's not clear why the skeleton has any endpoints at all in this case.  It's worth digging into that more to ensure it's robust.
- The perpendicular uses an arbitrary window to compute the local slope.  We could probably do better by averaging over several different window sizes.
- All this still needs to be wrapped up as part of a larger analysis loop, which finds perpendiculars for every contact and ensures that the endpoints are in the right cell (using the cell segmentation layer), and outputs those in NG or Precomputed format.